In [26]:
%matplotlib inline
import pandas as pd
import numpy as np
df = pd.read_csv('./new_train.csv', lineterminator='\n')
print(df.head(5))
doc_article = df['article']
doc_title = df['title']
doc_channel = df['channel']
doc_topics = df['topics'].fillna('')
doc_h2 = df['h2'].fillna('')
doc_author = df['author']
#df_val = pd.read_csv('./clean_val.csv')
#doc_val = df_val['article']

   Unnamed: 0     id  label  weekday  \
0           0   5655      1        0   
1           1  26310      1        0   
2           2  11734      1        0   
3           3  15709      1        1   
4           4   4207      0        0   

                                               title  links            author  \
0  32 Halloween Costumes for People Who Love the ...     39  Colin Gorenstein   
1  Samsung Comes Back Swinging After Apple's Big Day      8    Todd Wasserman   
2  Vine Challenge: Use the Newest Update to Creat...      5         Anne Park   
3  Google Launches Interactive Winter Wonderlabs ...      8    Adario Strange   
4  The Rock Tweets Details of Preposterous 'Hercu...      8         Sam Laird   

                                             article       channel  \
0  The year 2014 has been a big year for three-br...   watercooler   
1  Samsung has refrained from attacking Apple in ...      business   
2  Vine's latest update introduced a ton of new f...  social-m

In [7]:
import re
from bs4 import BeautifulSoup
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tokenizer(text):
    return re.split('\s+', text.strip())
#print(tokenizer('runners like running and thus they run'))

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]
#print(tokenizer_stem_nostop('runners like running and thus they run'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stanl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
doc_words = pd.concat([doc_title, doc_h2, doc_article])

In [35]:
print(doc_words.head(5))
print(doc_words.shape)

0    32 Halloween Costumes for People Who Love the ...
1    Samsung Comes Back Swinging After Apple's Big Day
2    Vine Challenge: Use the Newest Update to Creat...
3    Google Launches Interactive Winter Wonderlabs ...
4    The Rock Tweets Details of Preposterous 'Hercu...
dtype: object
(58050,)


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,1),
                        preprocessor=preprocessor,
                        tokenizer=tokenizer_stem_nostop)

tfidf.fit(doc_words)

top = 10
# get idf score of vocabularies
idf = tfidf.idf_
print('[vocabularies with smallest idf scores]')
sorted_idx = idf.argsort()

for i in range(top):
    print('%s: %.2f' %(tfidf.get_feature_names()[sorted_idx[i]], idf[sorted_idx[i]]))

D:\Program_files\Anaconda 3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


[vocabularies with smallest idf scores]
also: 2.21
see: 2.24
imag: 2.45
new: 2.54
one: 2.55
time: 2.63
like: 2.65
make: 2.75
year: 2.77
get: 2.82


In [36]:
doc_title_tfidf = tfidf.transform(doc_title)
doc_article_tfidf = tfidf.transform(doc_article)
doc_h2_tfidf = tfidf.transform(doc_h2)

In [54]:
#doc_h2_tfidf = doc_h2_tfidf.toarray()
doc_title_tfidf = doc_title_tfidf.toarray()

MemoryError: Unable to allocate 16.0 GiB for an array with shape (19350, 110867) and data type float64

In [43]:
from scipy import sparse
stack = sparse.hstack((doc_title_tfidf, doc_h2_tfidf))
print(stack.shape)

(19350, 221734)


In [64]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier(max_depth=5,learning_rate=0.015,objective="binary:logistic", eval_metric="error")
scores = cross_val_score(estimator=model, X=stack, y=df['label'], \
                         cv=5, scoring='roc_auc')
print('%.3f (+/-%.3f)' % (scores.mean(), scores.std()))

D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\Program_files\Anaconda 3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The u

0.533 (+/-0.006)


In [58]:
import scipy as sp
from sklearn.feature_extraction.text import HashingVectorizer

# hash words to 1024 buckets
hashvec = HashingVectorizer(n_features=2**10,
                            preprocessor=preprocessor,
                            tokenizer=tokenizer_stem_nostop)

# no .fit needed for HashingVectorizer, since it's defined by the hash function

# transform sentences to vectors of dimension 1024
doc_title_hash = hashvec.transform(doc_title)
doc_h2_hash = hashvec.transform(doc_h2)
doc_topic_hash = hashvec.transform(doc_topics)
stack = sparse.hstack((doc_title_hash, doc_h2_hash))

NameError: name 'doc_topic' is not defined

In [62]:
stack = sparse.hstack((doc_title_hash, doc_h2_hash))
stack = sparse.hstack((stack, doc_topic_hash))

In [60]:

doc_topic_hash = hashvec.transform(doc_topics)

In [9]:
#doc_tfidf_val = tfidf.transform(doc_val).toarray()
#doc_hash_val = hashvec.transform(doc_val)
#print(doc_tfidf_val.shape)
#print(doc_hash_val.shape)
#stack_val = sparse.hstack((doc_tfidf_val, doc_hash_val))
#print(stack_val.shape)

In [10]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
#y_train = df['label']
#model = xgb.XGBClassifier(max_depth=7,learning_rate=0.005,n_estimators=100,objective="binary:logistic", eval_metric="error")
#model.fit(stack,y_train)

#y_pred = model.predict(stack_val)
#accuracy = accuracy_score(df_val['label'],y_pred)
#print('accuarcy:%.2f%%'%(accuracy*100))

In [12]:
import pickle
filename = 'finalized_model.pkl'
#pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [13]:
df_test = pd.read_csv('./clean_test.csv')
doc_test = df_test['article']
doc_tfidf_test = tfidf.transform(doc_test).toarray()
doc_hash_test = hashvec.transform(doc_test)
print(doc_tfidf_test.shape)
print(doc_hash_test.shape)
stack_test = sparse.hstack((doc_tfidf_test, doc_hash_test))
print(stack_test.shape)

MemoryError: Unable to allocate 5.99 GiB for an array with shape (8292, 96925) and data type float64

In [ ]:
y_pred = loaded_model.predict_proba(stack_test)
for i in range(len(y_pred)):
    predict.append([])
    predict[i].append(df_test['Unnamed: 0'])
    predict[i].append(y_pred)
    
pred_y_out = pd.DataFrame(predict, columns=['Id', 'Popularity'])
pred_y_out.to_csv('test.csv', index=False)